In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

### ADJUST BRIGHTNESS <a class="anchor" id="c11"></a>

In [ ]:
def adjust_brightness(image_arr, level_brightness):
    img_result = image_arr.astype(np.int16) + level_brightness
    img_result = np.clip(img_result, 0, 255)
    return img_result.astype(np.uint8)

### ADJUST CONTRAST <a class="anchor" id="c11"></a>

In [ ]:
def adjust_contrast(image_arr, level_contrast):
    img_result = level_contrast*(image_arr.astype(np.float32) - 128) + 128
    img_result = np.clip(img_result, 0, 255)
    return img_result.astype(np.uint8)

### FLIP UPDOWN <a class="anchor" id="c11"></a>

In [ ]:
def flip_updown(image_arr):
    return np.flipud(image_arr)

### FLIP RIGHTLEFT <a class="anchor" id="c11"></a>

In [ ]:
def flip_rightleft(image_arr):
    return np.fliplr(image_arr)

### RGB TO GREY SCALE/ SEPIA <a class="anchor" id="c11"></a>


In [ ]:
def convert_gray_sepia(image_arr, method = 'grayscale'):
    if method == 'grayscale':
        return np.mean(image_arr.astype(np.float64), axis = 2).astype(np.uint8)
    elif method == 'sepia':
        image_arr = image_arr.astype(np.float64)
        R_scalar = np.array([0.393, 0.769, 0.189])
        G_scalar = np.array([0.349, 0.686, 0.168])
        B_scalar = np.array([0.272, 0.534, 0.131])
        copy_image = image_arr.copy()
        image_arr[:,:,0] = np.dot(copy_image, R_scalar)
        image_arr[:,:,1] = np.dot(copy_image,G_scalar)
        image_arr[:,:,2] = np.dot(copy_image,B_scalar)

        return np.clip(image_arr,0,255).astype(np.uint8)

### BLUR IMAGE WITH GAUSSIAN KERNEL<a class="anchor" id="c11"></a>


In [ ]:
def blur_image(image_arr):
    #save the boundary in the return image array
    new_image = np.ones_like(image_arr)
    gaussian_kernel = np.array([[1,4,6,4,1],
                                [4,16,24,16,4],
                                [6,24,36,24,6],
                                [4,16,24,16,4],
                                [1,4,6,4,1]])
    gaussian_kernel = gaussian_kernel/256
    if(image_arr.ndim == 3):
        for channel in range(3):
            new_image[:, :, channel] = convolution_blur(image_arr[:, :, channel], gaussian_kernel)
    
    else:
        new_image = convolution_blur(image_arr, gaussian_kernel)
    
    return np.clip(new_image,0,255)

def convolution_blur(channel_matrix, gausian_kernel):
    result_matrix = np.ones_like(channel_matrix)

    result_matrix[:1,:] = channel_matrix[:1,:]
    result_matrix[:-2,:] = channel_matrix[:-2,:]
    result_matrix[:,:1] = channel_matrix[:,:1]
    result_matrix[:,:-2] = channel_matrix[:, :-2]

    for i in range(2, channel_matrix.shape[0]-3):
        for j in range(2, channel_matrix.shape[1]-3):
            sub_arr = channel_matrix[i-2:i+3, j-2:j+3]
            result_matrix[i][j] = np.sum(sub_arr*gausian_kernel)

    return result_matrix

### SHARPEN IMAGE <a class="anchor" id="c11"></a>


In [ ]:
def sharpen_image(image_arr):
    #save the boundary in the return image array
    new_image = np.ones_like(image_arr)
    gaussian_kernel = np.array([[1,4,6,4,1],
                                [4,16,24,16,4],
                                [6,24,-476,24,6],
                                [4,16,24,16,4],
                                [1,4,6,4,1]])/-256
    if(image_arr.ndim == 3):
        for channel in range(3):
            new_image[:, :, channel] = convolution_sharpen(image_arr[:, :, channel], gaussian_kernel)
    
    else:
        new_image = convolution_sharpen(image_arr, gaussian_kernel)
    return np.clip(new_image,0,255)

def convolution_sharpen(channel_matrix, gausian_kernel):
    result_matrix = np.ones_like(channel_matrix)
    result_matrix[:1,:] = channel_matrix[:1,:]
    result_matrix[:-2,:] = channel_matrix[:-2,:]
    result_matrix[:,:1] = channel_matrix[:,:1]
    result_matrix[:,:-2] = channel_matrix[:, :-2]

    for i in range(2, channel_matrix.shape[0]-3):
        for j in range(2, channel_matrix.shape[1]-3):
            sub_arr = channel_matrix[i-2:i+3, j-2:j+3]
            result_matrix[i][j] = np.sum(sub_arr*gausian_kernel)

    return result_matrix

### CROP FROM CENTER <a class="anchor" id="c11"></a>


In [ ]:
def crop_center(image_arr, height, width):
    width = min(width, image_arr.shape[1])
    height = min(height, image_arr.shape[0])
    center = [image_arr.shape[1]//2, image_arr.shape[0]//2]
    first_corner = [center[0] - width//2, center[1] - height//2]
    return image_arr[first_corner[1] : first_corner[1] + height, first_corner[0]: first_corner[0] + width,:]

### CROP CIRCLE <a class="anchor" id="c11"></a>


In [ ]:
def create_circle_mask(image_arr):
    center_y = image_arr.shape[0]//2
    center_x = image_arr.shape[1]//2
    y, x = np.ogrid[0:image_arr.shape[0], 0:image_arr.shape[1]]
    radius = min(center_x, center_y)
    circle_mask = (x-center_x)**2 + (y-center_y)**2
    return circle_mask <= radius **2

In [ ]:
def crop_circle(image_arr):
    mask = create_circle_mask(image_arr)
    new_image = image_arr.copy()
    new_image[~mask] = 0
    return new_image

### CROP ELLIPSE <a class="anchor" id="c11"></a>


1. Crop a picture to square first:

In [ ]:
def crop_to_square(image_arr):
    side = min(image_arr.shape[0], image_arr.shape[1]) #the length side of square
    center = [image_arr.shape[1]//2, image_arr.shape[0]//2]
    first_corner = [center[0] - side//2, center[1] - side // 2]
    return image_arr[first_corner[1] : first_corner[1] + side, first_corner[0] : first_corner[0] + side, :]

2. Create a mask ellipse

In [ ]:
def create_mask_ellipse(image_arr, angle):
    side = min(image_arr.shape[0], image_arr.shape[1])
    center = [image_arr.shape[1]//2, image_arr.shape[0]//2]
    
    #major axis and minor axis is in form of power of 2

    major_axis = 0.4225* side**2
    minor_axis = 0.0775* side**2
    y, x = np.ogrid[:image_arr.shape[0], :image_arr.shape[1]]
 
    x_val = x - center[0]
    y_val = y - center[1]
    radian_angle = np.radians(angle)
    cos_val = np.cos(radian_angle)
    sin_val = np.sin(radian_angle)
    
    ellipse_domain = ((x_val*cos_val + y_val*sin_val)**2/major_axis) + ((x_val*sin_val - y_val*cos_val)**2/minor_axis)
    return ellipse_domain <= 1

3. Crop to Ellipse

In [ ]:
def crop_to_ellipse(image_arr):
    square_img = crop_to_square(image_arr)
    
    ellipse1 = create_mask_ellipse(square_img, 45)
    ellipse2 = create_mask_ellipse(square_img, 135)
    ellipse_combine = ellipse1 | ellipse2
    
    new_image = square_img.copy()
    new_image[~ellipse_combine] = 0
    return new_image

In [ ]:
def menu():
    print("---------------- MENU ----------------")
    print("0    - Do all features.")
    print("1    - Adjust the brightness.")
    print("2.   - Adjust the contrast.")
    print("3.   - Flip image.")
    print("4.   - Change RGB picture to Gray or Sepia picture.")
    print("5.   - Blur and sharpen image.")
    print("6    - Crop picture in Center.")
    print("7.   - Crop in Circle frame.")
    print("8.   - Crop in 2 intersecting Ellipse.")
    print("--------------------------------------")



In [ ]:
def convert_png_jpg(png_path,nameFile):
    try:
        img = Image.open(png_path)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        img.save(nameFile, 'JPEG')
        print(f"Image successfully converted to jpg")
    except Exception as e:
        print(f"Error converting the image: {e}")

In [ ]:
def read_Image_array(img_path):
    try:
        if img_path.split(".")[-1] == 'png': 
            convert_png_jpg(img_path, img_path.split(".")[0] + ".jpg")
            img_path = img_path.split(".")[0]+'.jpg'
            
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        return np.array(image)
    except Exception as e:
        print(f"Error while reading the image: {e}")
        return None

In [ ]:
def getNameImage(image_path):
    return image_path.split(".")[0]

In [ ]:
def exportImage(image_path, output_name, adjust_picture): 
    if adjust_picture.ndim == 3:
        mode = 'RGB'
    else:
        mode = 'L'  # 'L' stands for Grayscale

    file_extension = image_path.split(".")[-1]
    output_path = f"./{output_name}.{file_extension}"

    try:
        im = Image.fromarray(adjust_picture, mode=mode)
        im.save(output_path)
        print(f"Image saved as {output_path}.")
    except Exception as e:
        print(f"An error occurred while saving the image: {e}")

In [ ]:
def main():
    #get input picture and get their array
    menu()
    img_path = input("Enter your image path: ")
    image_arr = read_Image_array(img_path)
    
    
    choice = input("Enter your choice: ")
    if choice == '0':
        #bright
        bright_picture = adjust_brightness(image_arr, 50)
        exportImage(img_path, getNameImage(img_path) + "_bright", bright_picture)
        
        #constrast
        contrast_picture = adjust_contrast(image_arr, 1.5)
        exportImage(img_path, getNameImage(img_path) + "_contrast", contrast_picture)
        
        #flip updown
        flip_ud = flip_updown(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_updown", flip_ud)
        
        #flip rightleft
        flip_rl = flip_rightleft(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_rightleft", flip_rl)
        
        #Change to Gray
        gray_pic = convert_gray_sepia(image_arr, method = 'grayscale')
        exportImage(img_path, getNameImage(img_path) + "_gray", gray_pic)
        
        #Change to Sepia
        sepia_pic = convert_gray_sepia(image_arr, method = 'sepia')
        exportImage(img_path, getNameImage(img_path) + "_sepia", sepia_pic)
        
        #Blur image
        blur_img = blur_image(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_blur", blur_img)
        
        #sharpen image 
        sharpen_img = sharpen_image(blur_img)
        exportImage(img_path, getNameImage(img_path) + "_sharpen", sharpen_img)

        #crop center
        center_pic = crop_center(image_arr,200,200)
        exportImage(img_path, getNameImage(img_path) + "_center", center_pic)

        #crop circle
        circle_pic = crop_circle(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_circle", circle_pic)

        #crop to ellipse
        ellipse_pic = crop_to_ellipse(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_ellipse", ellipse_pic)
        
    elif choice == '1':
        bright_picture = adjust_brightness(image_arr, 50)
        exportImage(img_path, getNameImage(img_path) + "_bright", bright_picture)
        plt.imshow(bright_picture)

    elif choice == '2':
        contrast_picture = adjust_contrast(image_arr, 1.5)
        exportImage(img_path, getNameImage(img_path) + "_contrast", contrast_picture)
        plt.imshow(contrast_picture)

    elif choice == '3':
        print("1. Flip updown")
        print("2. Flip rightleft")
        subchoice = input("Your choice: ")
        if subchoice == '1':
            flip_ud = flip_updown(image_arr)
            exportImage(img_path, getNameImage(img_path) + "_updown", flip_ud)
            plt.imshow(flip_ud)
        elif subchoice == '2':
            flip_rl = flip_rightleft(image_arr)
            exportImage(img_path, getNameImage(img_path) + "_rightleft", flip_rl)
            plt.imshow(flip_rl)
        else:
            print("Wrong input!")

    elif choice == '4':
        #Change to Gray
        gray_pic = convert_gray_sepia(image_arr, method = 'grayscale')
        exportImage(img_path, getNameImage(img_path) + "_gray", gray_pic)
        plt.figure(figsize=(12, 12))
        plt.subplot(2,2,1)
        plt.imshow(gray_pic, cmap = 'gray')
        plt.title("Gray Image",size = 9)

        #Change to Sepia
        sepia_pic = convert_gray_sepia(image_arr, method = 'sepia')
        exportImage(img_path, getNameImage(img_path) + "_sepia", sepia_pic)
        plt.subplot(2,2,2)
        plt.imshow(sepia_pic)
        plt.title("Sepia Image",size = 9)
    
    elif choice == '5':
        #Blur image
        blur_img = blur_image(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_blur", blur_img)

        #Sharpen image
        sharpen_img = sharpen_image(blur_img)
        exportImage(img_path, getNameImage(img_path) + "_sharpen", sharpen_img)
        
    elif choice == '6':
        center_pic = crop_center(image_arr,200,200)
        exportImage(img_path, getNameImage(img_path) + "_center", center_pic)
        plt.imshow(center_pic)

    elif choice == '7':
        circle_pic = crop_circle(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_circle", circle_pic)
        plt.imshow(circle_pic)

    elif choice == '8':
        ellipse_pic = crop_to_ellipse(image_arr)
        exportImage(img_path, getNameImage(img_path) + "_ellipse", ellipse_pic)
        plt.imshow(ellipse_pic)
    else:
        print("Wrong input!")
  
if __name__ == "__main__":
    main()
